### Bicycle Workshop (Python)

Suppose `B` bicycle riders live alternating between riding a bicycle and repairing it at a workshop. Due to capacity limitations, the workshop can accommodate most `C` repairs at a time. The algorithm below ensures this by using a *counting semaphore* initialized to `C`.

```algorithm

var shop: semaphore = C

process rider(i: 0 .. B – 1)
    while true do
        ride bicycle
        P(shop)
        repair bicycle
        V(shop)
```

This is a generalization of the critical section problem in that up to `C` processes can be in the critical section, `repair bicycle`. Assuming that the semaphore implementation is weakly fair, which of the four properties of critical sections (mutual exclusion, no deadlock, no unnecessary delay, eventual entry) hold?

YOUR ANSWER HERE

*Answer:*  
- Mutual exclusion: no, as up to `C` riders can be in the shop.
- No deadlock: yes, if two or more riders are trying to enter and no other rider is in the shop, one rider will enter.
- No unnecessary delay: yes, if one rider is trying to enter the shop and no other rider is in the shop, the rider is not prevented.
- Eventual entry: no, even though the semaphore will repeatedly allow a rider to enter the critical section, under weak fairness, the scheduler may select a rider each time they are blocked.

Here is an implementation in Python that adds output and limits the riders to `4` rounds of riding and repairing:

In [ ]:
from threading import Thread, Semaphore
from time import sleep
from sys import stdout

class Rider(Thread):
    def __init__(self, i):
        Thread.__init__(self); self.i = i
    def run(self):
        for _ in range(4):
            stdout.write(str(self.i) + ' riding\n'); sleep(2)
            shop.acquire()
            stdout.write(str(self.i) + " repairing\n"); sleep(1)
            shop.release()
        stdout.write(str(self.i) + " done\n")

C, B = 3, 10
shop = Semaphore(C)
riders = {Rider(i) for i in range(B)}
for r in riders: r.start()
for r in riders: r.join()

Modify the implementation to display the number of bicycles in the shop! That number is initially `0` and changes each time a bicycle enters or exits the shop.

In [ ]:
YOUR CODE HERE

In [ ]:
from threading import Thread, Semaphore
from time import sleep
from sys import stdout

class Rider(Thread):
    def __init__(self, i):
        Thread.__init__(self); self.i = i
    def run(self):
        for _ in range(4):
            global occ
            stdout.write(str(self.i) + ' riding\n'); sleep(2)
            shop.acquire()
            mutex.acquire(); occ += 1; stdout.write("occupancy: " + str(occ) + "\n"); stdout.flush(); mutex.release()
            stdout.write(str(self.i) + " repairing\n"); sleep(1)
            mutex.acquire(); occ -= 1; stdout.write("occupancy: " + str(occ) + "\n"); mutex.release()
            shop.release()
        stdout.write(str(self.i) + " done\n")

C, B = 3, 10
shop, mutex, occ = Semaphore(C), Semaphore(1), 0
stdout.write("occupancy: 0\n")
riders = {Rider(i) for i in range(B)}
for r in riders: r.start()
for r in riders: r.join()

Is the output always consistent in that the occupancy always corresponds to the number of riders who have entered the shop minus the number of riders who have left the shop?

YOUR ANSWER HERE

*Answer:*  
In the above solution, each thread writes to its buffer. There is no guarantee when the buffers are flushed and that the output appears on the screen immediately. This could be made more likely by explicitly flushing the buffer by inserting `stdout.flush()`, though there is no guarantee that `flush()` will be called right after `write()` in a thread, so the output may still appear inconsistent. A way to get consistent output is if each thread atomically appends its output to a string, and that string is then displayed, or equivalently, if writing and flushing the buffer is made atomic through a mutex semaphore.

A *binary semaphore* is a semaphore that takes only two values, `0` and `1`. (Binary semaphores are sometimes called locks or mutexes.) Give an algorithm for the bicycle workshop equivalent to the above one but using only binary semaphores! It can use additional integer variables, but minimize the number of variables you use!

```algorithm
YOUR ALGORITHM HERE
```

*Answer:*  
The idea here is to introduce an integer variable, `avail`, for the number of available spots in the workshop; it is incremented on each attempted entry and thus can become negative. When negative, its absolute value is the number of riders waiting for entry. One semaphore, `mutex`, ensures that updates to `avail` are atomic. Another semaphore, `free`, is used to notify waiting riders when a spot becomes available.

```algorithm
var mutex: semaphore = 1
var free: semaphore = 0
var avail: integer = C

process rider(i: 0 .. B – 1)
    while true do
        ride bicycle
        P(mutex); avail := avail – 1
        if avail < 0 then V(mutex); P(free)
        else V(mutex)
        repair bicycle
        P(mutex); avail := avail + 1
        if avail <= 0 then V(mutex); V(free)
        else V(mutex)
```

Does your algorithm guarantee the four properties of critical sections?

YOUR ANSWER HERE

*Answer:*  
- Mutual exclusion: no, as up to `C` riders can be in the shop.
- No deadlock: yes, if two or more riders are trying to enter and no other rider is in the shop, one rider will enter.
- No unnecessary delay: yes, if one rider is trying to enter the shop and no other rider is in the shop, the rider is not prevented.
- Eventual entry: no, even though the semaphore will repeatedly allow a rider to pass `P(mutex)` and `P(free)`, under weak fairness, the scheduler may select a rider each time they are blocked.

Implement your algorithm in Python! You may add output to display the activity of each rider and the shop occupancy.

In [ ]:
YOUR CODE HERE

In [ ]:
from threading import Thread, Semaphore
from time import sleep
from sys import stdout

class Rider(Thread):
    def __init__(self, i):
        Thread.__init__(self); self.i = i
    def run(self):
        for _ in range(4):
            global avail
            stdout.write(str(self.i) + ' riding\n'); sleep(2)
            mutex.acquire(); avail -= 1; stdout.write("avail after enter: " + str(avail) + "\n")
            if avail < 0: mutex.release(); free.acquire()
            else: mutex.release()
            stdout.write(str(self.i) + " repairing\n"); sleep(1)
            mutex.acquire(); avail += 1; stdout.write("avail after exit: " + str(avail) + "\n")
            if avail <= 0: mutex.release(); free.release()
            else: mutex.release()
        stdout.write(str(self.i) + " done\n")

C, B = 3, 10
mutex, free, avail = Semaphore(1), Semaphore(0), C
stdout.write("occupancy: 0\n")
riders = {Rider(i) for i in range(B)}
for r in riders: r.start()
for r in riders: r.join()